In [94]:
import numpy as np
import pandas as pd
import os

In [95]:
models = ['LSTM', 'RNN']
date_list = [['2001-01-01', '2020-01-01', '2022-01-02', '2024-01-05'], ['1990-01-05', '2010-01-05', '2023-01-05', '2024-01-05'], ['2000-01-05', "2024-01-05", "2024-01-05", "2025-01-05"], ['2005-01-05', '2010-01-05', '2020-01-05', '2024-01-05']]

In [96]:
df = pd.DataFrame({"ETF":[],"Metric":[],"Final_Value":[],"Passive_Potential":[],"Relative_MSE":[],"MAPE":[],"DM_stat":[],"DM_p":[], "model_type":[], "date_config":[]})

In [97]:
for i in range(4):
    for m in models:
        d = pd.read_csv(f"../output/runtest_10027[final]/date_config{i}/{m}/overall_results.csv")
        d['model_type'] = m
        d['date_config'] = f'train: start {date_list[i][0]}, end {date_list[i][1]}; test: start {date_list[i][2]}, end {date_list[i][3]}'
        df = pd.concat([df, d], axis=0, ignore_index=True)



In [98]:
df

,ETF,Metric,Final_Value,Passive_Potential,Relative_MSE,MAPE,DM_stat,DM_p,model_type,date_config
0,XLB,ATR,1000.000000,1030.095425,0.003427,0.356005,NaN,NaN,LSTM,"train: start 2001-01-01, end 2020-01-01; test:..."
1,XLB,SD_Squared_Returns,1081.621177,1030.095425,0.002011,0.300276,59.845737,0.000000e+00,LSTM,"train: start 2001-01-01, end 2020-01-01; test:..."
2,XLB,SD_Prices,1051.835579,1030.095425,0.001454,0.313790,11.584096,4.961736e-31,LSTM,"train: start 2001-01-01, end 2020-01-01; test:..."
3,XLE,ATR,835.254085,1257.954208,0.000133,0.051967,NaN,NaN,LSTM,"train: start 2001-01-01, end 2020-01-01; test:..."
4,XLE,SD_Squared_Returns,840.061493,1257.954208,0.000361,0.127410,23.677640,6.129623e-124,LSTM,"train: start 2001-01-01, end 2020-01-01; test:..."
...,...,...,...,...,...,...,...,...,...,...
211,XLY,SD_Squared_Returns,1000.000000,1527.925532,0.002535,0.954616,60.647165,0.000000e+00,RNN,"train: start 2005-01-05, end 2010-01-05; test:..."
212,XLY,SD_Prices,1000.000000,1527.925532,0.003130,0.866822,3.123971,1.784282e-03,RNN,"train: start 2005-01-05, end 2010-01-05; test:..."
213,XLU,ATR,1033.405108,1022.748966,0.000841,0.364591,NaN,NaN,RNN,"train: start 2005-01-05, end 2010-01-05; test:..."
214,XLU,SD_Squared_Returns,1066.251352,1022.748966,0.000518,0.448898,48.278011,0.000000e+00,RNN,"train: start 2005-01-05, end 2010-01-05; test:..."


In [99]:
import pandas as pd

# 1. Drop the unwanted columns
df_clean = df.drop(columns=["DM_stat", "DM_p"])

# 2. Define the columns to include and aggregate (mean if duplicates)
cols = ["Final_Value", "Passive_Potential", "Relative_MSE", "MAPE"]
group_keys = ["Metric", "model_type", "date_config", "ETF"]

# 3. Aggregate
agg = (
    df_clean
    .groupby(group_keys, as_index=False)[cols]
    .mean()
)

# 4. Format the display string in one box per ETF combination
def make_box(r):
    return (
        f"FV: {r['Final_Value']:.3f}; PP: {r['Passive_Potential']:.3f}; RelMSE: {r['Relative_MSE']:.3f}; MAPE: {r['MAPE']:.3f}"
    )

agg["display"] = agg.apply(make_box, axis=1)

# 5. Pivot to wide format: index = Metric, model_type, date_config; columns = ETF; values = display box
wide_display = (
    agg
    .pivot_table(
        index=["Metric", "model_type", "date_config"],
        columns="ETF",
        values="display",
        aggfunc="first"  # already aggregated
    )
)

# 6. (Optional) Make it presentation-friendly by flattening the column index if needed
wide_display = wide_display.rename_axis(columns=None)  # remove name 'ETF' from columns

# 7. Export or inspect
wide_display.to_csv("final_wide_display_table.csv")  # for slides
print(wide_display.head(8).to_string())


                                                                                                                                                      XLB                                                     XLE                                                     XLF                                                     XLI                                                     XLK                                                     XLP                                                     XLU                                                     XLV                                                     XLY
Metric model_type date_config                                                                                                                                                                                                                                                                                                                                                                                 

In [100]:
df_n

model_type,LSTM,RNN
ETF,,
XLB,1138.416,1023.681
XLE,1113.255,1070.502
XLF,1167.644,1100.818
XLI,1059.143,1022.434
XLK,1110.520,1027.164
XLP,1091.426,1061.929
XLU,1074.359,1054.539
XLV,1036.190,1012.566
XLY,1196.586,1079.946
